In [1]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[:4]
    #cwd.append('loaded')
    cwd='/'.join(cwd)
    return cwd
path=get_algo_path()+'/loaded/Z normalised/ANN.py'
path

'C:/Users/Omar/00_Bank_loan_project/loaded/Z normalised/ANN.py'

In [2]:
exec(open(path).read())



Train on 71786 samples, validate on 30766 samples
Epoch 1/200
71786/71786 [==============================] - 12s 164us/sample - loss: 0.6733 - accuracy: 0.5611 - val_loss: 0.5949 - val_accuracy: 0.6854
Epoch 2/200
71786/71786 [==============================] - 11s 147us/sample - loss: 0.5744 - accuracy: 0.6764 - val_loss: 0.5143 - val_accuracy: 0.7176
Epoch 3/200
71786/71786 [==============================] - 3s 36us/sample - loss: 0.5254 - accuracy: 0.6926 - val_loss: 0.4830 - val_accuracy: 0.7334
Epoch 4/200
71786/71786 [==============================] - 10s 137us/sample - loss: 0.5053 - accuracy: 0.7032 - val_loss: 0.4715 - val_accuracy: 0.7394
Epoch 5/200
71786/71786 [==============================] - 3s 46us/sample - loss: 0.4927 - accuracy: 0.7143 - val_loss: 0.4652 - val_accuracy: 0.7448
Epoch 6/200
71786/71786 [==============================] - 10s 145us/sample - loss: 0.4863 - accuracy: 0.7176 - val_loss: 0.4593 - val_accuracy: 0.7503
Epoch 7/200
71786/71786 [=================